# Is there a Vélo Toulouse station near you ?

Application of [OAlley API](https://api.oalley.fr) to create a map of coverage area of Vélo Toulouse stations (city bike rental).

Algorithm:
1. Fetch a list of all GPS coordinates of Velo Toulouse stations.
2. For each station, compute a 5 minutes by foot isochrone with [OAlley API](https://api.oalley.fr) .
3. Using `shapely`, perform an union on all requested zones to compute the final zone.
4. Draw with folium

In [2]:
import folium
import grequests
import requests
import polyline
from urllib.parse import urlencode
import json
from shapely.geometry.polygon import Polygon
from shapely.ops import cascaded_union

baseurl = "https://api.oalley.fr/api/AppKeys/"
method = "/isochronous?"

appkey = "YOUR-API-KEY"


## Get the list of stations


In [3]:
# Request bike stations
r = requests.get("https://data.toulouse-metropole.fr/explore/dataset/velo-toulouse/download/?format=json&timezone=Europe/Berlin")

# Extract coords for each
coords = [{'lat': s['fields']['geo_shape']['coordinates'][1], 'lng': s['fields']['geo_shape']['coordinates'][0], 'duration': 0} for s in r.json()]

coords_5m = [dict(c, duration = 60) for c in coords]

print("Found %d stations" % len(coords))

zones = []

i = 0

# Called when OAlley returned a zone
def callback(res, **kwargs):
    global i
    if(res.status_code != 200): 
        return print(res.json()) 
    
    body = res.json()
    zone = polyline.decode(body['polyline'])
    zones.append(zone)
    i += 1
    if(i % 10 == 0):
        print("%d / %d" % (i, len(coords)))
    
def exception_handler(request, exception):
    print("Error : %s" % exception)
    
    
# Prepare all requests for OAlley
urls = [baseurl + appkey + method + urlencode(point) for point in coords_5m]
reqs = [grequests.get(url, callback=callback) for url in urls]

# Execute all requests
grequests.map(reqs, exception_handler=exception_handler)

print("Done !")

Found 282 stations
10 / 282
20 / 282
30 / 282
40 / 282
50 / 282
60 / 282
70 / 282
80 / 282
90 / 282
100 / 282
110 / 282
120 / 282
130 / 282
140 / 282
150 / 282
160 / 282
170 / 282
180 / 282
190 / 282
200 / 282
210 / 282
220 / 282
230 / 282
240 / 282
250 / 282
260 / 282
270 / 282
280 / 282
Done !


In [4]:
# Merge all zones
polygons = [Polygon(z) for z in zones]

nonvalids = [p for p in polygons if not p.is_valid]

# Some zones have issues for shapely. Ignore them.
print("Found %d invalid polygons" % len(nonvalids))
    
polygons = [p for p in polygons if p.is_valid]
    
allzones = cascaded_union(polygons)

print("Merged %d into %d" % (len(polygons),len(allzones)))

Self-intersection at or near point 43.567167048775872 1.4743002789902928
Self-intersection at or near point 43.600247913446672 1.4291103709428141
Self-intersection at or near point 43.604723501567754 1.431949080503192
Self-intersection at or near point 43.606791538461536 1.4477015384615408
Self-intersection at or near point 43.59552 1.4734400000000001
Self-intersection at or near point 43.606101335043583 1.4346724742807777
Self-intersection at or near point 43.610334097623635 1.4380124405908812
Self-intersection at or near point 43.620452172088143 1.4081875865687303
Self-intersection at or near point 43.601584781184179 1.4343496208752631
Self-intersection at or near point 43.609808974289216 1.4332129179703459
Self-intersection at or near point 43.604900000000001 1.4459500000000001
Self-intersection at or near point 43.608020247933901 1.4384253719007942
Self-intersection at or near point 43.592904999999995 1.4105300000000336
Self-intersection at or near point 43.632690367330994 1.450377

Found 54 invalid polygons
Merged 228 into 5


# Is there a Vélo Toulouse station near you ? The map.

Area in blue represents where you can find a Vélo Toulouse station for less than 5 minutes by walk.


# Une station de Vélo toulouse à moins de 5 minutes a pied ? Voici la carte.

La zone en bleu représente la zone ou vous pouvez trouver une station de Vélo Toulouse à moins de 5 minutes a pied (approximativement).

Réalisé avec [OAlley.fr](https://api.oalley.fr)

In [5]:
# Build output map
m = folium.Map(location=[46, 2], zoom_start=5)

contour_color = "#ffffff00" 
fill_color    = "#002d5f"
fill_color2    = "#03005f"

# Draw all computed zones on the map
for zone in allzones:
    pts = (list(zone.exterior.coords))
    folium.features.PolygonMarker(locations=pts, color=contour_color, fill_color=fill_color,  weight=1).add_to(m)

# Print the result
m.fit_bounds(m.get_bounds())

# IPython trick to display the map
m

In [6]:
# Eventually, dump all zones to a google polyline format if you want to display it on a website
arr = [ polyline.encode(list(z.exterior.coords)) for z in allzones]

json.dumps(arr)

'["ehdiGuhpGcAiCcAbD]nCdAfEQ@M?eLy@oLxBCDhBkPScGhB`A\\\\E|GTfHqEwCq@kGTlG_JBwEHoFcPjPu@aAeAqCmBnI]AeEu@uB`AUHz@fCFnAuGdClCdD@?_DzL|B|B~EGbBxFhEp@PyAxON|BN|@d@jC_Ad@}B|@zC|@zGtEdJv@gGtJvHfDwDjF}@cPgMhDyBqBeCZqAgDmBk@aATgCdDcQ{IjM}Ai@yBhC", "wkfiGs~oGi@mCqU}n@~Ixe@uK~@|EtB[xAwHJ`AlIi@xDoCjDcBtJf_@aT?N`AmAXAZA\\\\?X?r@J@@D?HCHQjLQqLi@vBcBC@kCnA_@By@y@ESEO", "{meiG}fsG{GuMaB~FyAnCdAbIkAHzEdGFj@qIjE_JrMVdFpHwBxGdCj@vCrHmBfBwBbCoCn@_DvBoAn@s@oA{DQ]fI_Cu@yA^uE`@mD{C{BuAk@wEx@uCUmBO", "ax_iGqplG{@pAiBjC_@n@a@j@yVaJXxC}C~HaAZ@@JHRLZP??VP`AbIB`DnLgDd@dKPAxACfJlCxEdDtBxAnChB`KARsJbG}@dAXcDwQj@mB`@{@`Au@vA}@f@aAlBc@jGcD|GcEeOMlA_Iw@UqCN}CS\\\\eIkAGxAkGiBkMRs@x@AzBnMvHuQn@VvQfOiEiSoCqJpD_Dc@kB[wBfPgGoD}EkCaCwBp@qQqEd@aHkBhGw`@_KnCoADAti@}Rt@j@qBzAzCZb@j@h@?}ArJ?jTxF{B`Fq@b@aHp@yIdCiBXe@vDt@_C_GFoBcGk@lBsBnAwCeDHUQMe@dDeTcGxEiD|AoBkEHuAKC_@Mi@iAx@eCzAqHNOpAkH|FyAzAd@lA\\\\hAwAqA{DlBaCB@MGeC_B@Yn@c@f@yCf@kAlHpGgBlCYPvHjBGFmElE`AvAfAzAnBb@bCaBrAlCjD~FfCe@{AxC`A~AzBhAlBsF|ElBRv@?CJq@`GzG?eG{AcE\\\\mAqA